<a href="https://colab.research.google.com/github/Srividhyak2011/Demo-Datascienceproject/blob/main/pegasus_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00


In [4]:
!pip install transformers[torch]

In [16]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

# Load PEGASUS model and tokenizer
model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Prepare your email dataset
emails_input = [
    "Subject: Greetings from XYZ Corp\n\nDear Customer, We're excited to offer you...",
    "Subject: Regarding Your Recent Purchase\n\nDear [Name], Thank you for purchasing...",
    # Add more email examples here...
]

# Tokenize the emails
tokenized_emails = tokenizer(emails_input, return_tensors="pt", padding=True, truncation=True)

# Fine-tuning the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
batch_size = 4  # Set your desired batch size

# Example fine-tuning loop
for epoch in range(3):  # 3 epochs as an example
    for i in range(0, len(tokenized_emails["input_ids"]), batch_size):
        batch_input_ids = tokenized_emails["input_ids"][i:i + batch_size]

        outputs = model(input_ids=batch_input_ids, labels=batch_input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_email_pegasus")
tokenizer.save_pretrained("./fine_tuned_email_pegasus")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('./fine_tuned_email_pegasus/tokenizer_config.json',
 './fine_tuned_email_pegasus/special_tokens_map.json',
 './fine_tuned_email_pegasus/spiece.model',
 './fine_tuned_email_pegasus/added_tokens.json')

In [17]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the fine-tuned model
fine_tuned_model = PegasusForConditionalGeneration.from_pretrained("/content/fine_tuned_email_pegasus")
tokenizer = PegasusTokenizer.from_pretrained("/content/fine_tuned_email_pegasus")

# Example prompt for generating an email
prompt = "Subject: Follow-up on our recent conversation\n\nDear [Name], I wanted to follow up..."

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate the email
output = fine_tuned_model.generate(input_ids, max_length=200, num_return_sequences=1)

# Decode and print the generated email
generated_email = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_email)


Subject: Follow-up on our recent conversation Dear [Name], I wanted to follow up...


In [36]:
# Training dataset
import pandas as pd
data = pd.read_csv('/EMAIL-DATASET.csv')
#train_emails_input = [
 #   "Subject: Greetings from XYZ Corp\n\nDear Customer, We're excited to offer you...",
  #  "Subject: Regarding Your Recent Purchase\n\nDear [Name], Thank you for purchasing...",
    # Add more email examples here for training...
#]
train_emails_input = data['input'].tolist()
#train_emails_target = [
#    "Dear Customer, We're excited to offer you our latest products...",
#    "Thank you for purchasing from us. Your order details...",
#    # Add corresponding target sequences for each input email here for training...
#]
train_emails_target = data['target'].tolist()
# Testing dataset
test_emails_input = [
    "Subject: Inquiry about Product Availability\n\nDear Team, I wanted to inquire...",
    "Subject: Feedback on Recent Service\n\nDear [Name], I wanted to share my feedback...",
    # Add more email examples here for testing...
]

test_emails_target = [
    "Dear Team, Could you provide information on the availability of...",
    "Dear [Name], Thank you for your feedback. We appreciate...",
    # Add corresponding target sequences for each input email here for testing...
]


In [37]:
from transformers import PegasusTokenizer

# Load PEGASUS tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

# Tokenize the training and testing emails
tokenized_train_emails = tokenizer(train_emails_input, text_pair=train_emails_target, return_tensors="pt", padding=True, truncation=True)
tokenized_test_emails = tokenizer(test_emails_input, text_pair=test_emails_target, return_tensors="pt", padding=True, truncation=True)


In [38]:
from transformers import PegasusForConditionalGeneration
import torch

# Load PEGASUS model
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')

# Prepare input and target sequences for training
train_input_ids = tokenized_train_emails["input_ids"]
#train_labels = tokenized_train_emails["labels"]

# Fine-tuning parameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
batch_size = 4
epochs = 3

# Fine-tuning loop
for epoch in range(epochs):
    for i in range(0, len(train_input_ids), batch_size):
        batch_input_ids = train_input_ids[i:i + batch_size]
        #batch_labels = train_labels[i:i + batch_size]

        outputs = model(input_ids=batch_input_ids, labels=batch_input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_email_pegasus")
tokenizer.save_pretrained("./fine_tuned_email_pegasus")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# Assuming you have test_emails_input and test_emails_target as before...

# Tokenize the test dataset
tokenized_test_emails = tokenizer(test_emails_input, return_tensors="pt", padding=True, truncation=True)

# Prepare input sequences for testing
test_input_ids = tokenized_test_emails["input_ids"]

# Evaluation loop
for i in range(len(test_input_ids)):
    input_seq = test_input_ids[i:i + 1]  # Take one input sequence
    output = model.generate(input_seq, max_length=200, num_return_sequences=1)

    # Decode the generated output
    generated_email = tokenizer.decode(output[0], skip_special_tokens=True)

    # Compare with the expected target
    expected_target = test_emails_target[i]

    # Print the generated email and the expected target
    print(f"Generated Email: {generated_email}")
    print(f"Expected Target: {expected_target}")
    print("----------------------------------------")


Generated Email: Subject: Inquiry about Product Availability Dear Team, I wanted to inquire...
Expected Target: Dear Team, Could you provide information on the availability of...
----------------------------------------
Generated Email: Subject: Feedback on Recent Service Dear [Name], I wanted to share my feedback...
Expected Target: Dear [Name], Thank you for your feedback. We appreciate...
----------------------------------------


In [40]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the fine-tuned model
model = PegasusForConditionalGeneration.from_pretrained("./fine_tuned_email_pegasus")
tokenizer = PegasusTokenizer.from_pretrained("./fine_tuned_email_pegasus")

# Get user input
user_input = input("Dear Team, I want to buy")

# Tokenize the user input
input_ids = tokenizer.encode(user_input, return_tensors='pt')

# Generate the email based on user input
output = model.generate(input_ids, max_length=200, num_return_sequences=1)

# Decode and print the generated email
generated_email = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Email:")
print(generated_email)


Dear Team, I want to buyDear Team, I want to buy
Generated Email:
Dear Team, I want to buy
